In this notebook we will test the candidate generation module 

In [7]:
from src.llm.llm_interface import UnifiedLLMInterface
# IMPORTs
from utils.task import Task
import json
import os
from src.pipeline.candidate_generation import candidate_generation
from dotenv import load_dotenv
from utils.prompt import load_prompt
import tiktoken
from tqdm import tqdm

load_dotenv()

True

In [8]:
# Function to load JSON data
def load_json_data(filepath):
    with open(filepath, 'r') as file:
        data = json.load(file)
    return data


# Function to create task object
def create_task(example):
    return Task(example)

In [9]:
# load the task data
filepath = "C:\\Users\yousf\Bureau\ConvergenceAI\CHESS_Impl\data\\subsampled_dev_set\\sub_sampled_bird_dev_set.json"
data = load_json_data(filepath)
# load the retrieved entities
filepath_entities = "C:/Users\yousf\Bureau\ConvergenceAI\CHESS_Impl\data/test/retrieved_entities.json"
retrieved_entities = load_json_data(filepath_entities)
# load the retrieved context
filepath_context = "C:/Users\yousf\Bureau\ConvergenceAI\CHESS_Impl\data/test/retrieved_context.json"
retrieved_context = load_json_data(filepath_context)
# load the selected schema
filepath_schema = "C:/Users\yousf\Bureau\ConvergenceAI\CHESS_Impl\data/test/selected_schema.json"
selected_schema = load_json_data(filepath_schema)

llm = UnifiedLLMInterface()

In [14]:
# test candidate generation module with a random sample
index = 22
example = data[index]
task = create_task(example)
entity_retrieval = {
    "similar_columns": {
        "bond": [
            "bond_type"
        ]
    },
    "similar_values": {}
}
context_retrieval = {
    "schema_with_descriptions": {
        "bond": {
            "bond_type": {
                "column_name": "",
                "column_description": "type of the bond",
                "value_description": "\u0093-\u0094: single bond '=': double bond '#': triple bond"
            },
            "molecule_id": {
                "column_name": "",
                "column_description": "identifying the molecule in which the bond appears",
                "value_description": ""
            },
            "bond_id": {
                "column_name": "",
                "column_description": "unique id representing bonds",
                "value_description": "TRxxx_A1_A2: TRXXX refers to which molecule A1 and A2 refers to which atom"
            }
        },
        "connected": {
            "bond_id": {
                "column_name": "bond id",
                "column_description": "bond id representing bond between two atoms",
                "value_description": ""
            }
        },
        "atom": {
            "molecule_id": {
                "column_name": "molecule id",
                "column_description": "identifying the molecule to which the atom belongs",
                "value_description": "TRXXX_i represents ith atom of molecule TRXXX"
            }
        }
    }
}
schema = {
    "bond": [
        "bond_type",
        "molecule_id"
    ],
    "molecule": [
        "molecule_id"
    ]
}

model = "llama-3"
ans = candidate_generation(task=task, retrieved_entities=entity_retrieval,
                           retrieved_context=context_retrieval,
                           selected_schema=schema,
                           llm=llm,
                           model=model,
                           num_samples=1)
ans

{
    "chain_of_thought_reasoning": "First, I identified the relevant table as 'bond' since it contains information about bond types. Then, I focused on the 'bond_type' column, which has an example value '#', indicating a triple bond. Next, I realized that I need to count the number of compounds that form a triple bond and then calculate the percentage. To do this, I joined the 'bond' table with the 'olecule' table to get the total number of compounds. Finally, I used the COUNT and SUM functions to calculate the percentage.",
    "SQL": "SELECT ROUND(COUNT(T1.bond_id) / (SELECT COUNT(*) FROM T1) * 100, 2) AS percentage FROM bond T1 WHERE T1.bond_type = '#';"
}


{'chain_of_thought_reasoning': "First, I identified the relevant table as 'bond' since it contains information about bond types. Then, I focused on the 'bond_type' column, which has an example value '#', indicating a triple bond. Next, I realized that I need to count the number of compounds that form a triple bond and then calculate the percentage. To do this, I joined the 'bond' table with the 'olecule' table to get the total number of compounds. Finally, I used the COUNT and SUM functions to calculate the percentage.",
 'SQL': "SELECT ROUND(COUNT(T1.bond_id) / (SELECT COUNT(*) FROM T1) * 100, 2) AS percentage FROM bond T1 WHERE T1.bond_type = '#';"}

In [5]:
#test candidate generation module with the subsampled test
model = "gpt-4"
num_samples = 1
res = []
for index, sample in tqdm(enumerate(data)):
    task = create_task(sample)
    ans = candidate_generation(task=task, retrieved_entities=retrieved_entities[index],
                               retrieved_context=retrieved_context[index],
                               selected_schema=selected_schema[index]["selected_schema"],
                               model=model,
                               num_samples=num_samples)
    res.append(ans)
    print(str(index) + " : ", ans)

1it [00:11, 11.06s/it]

0 :  {'chain_of_thought_reasoning': "To answer the question, I need to perform two tasks: 1) Identify the atom IDs that belong to the molecule 'TR346'. This can be achieved by querying the 'atom' table where 'molecule_id' equals 'TR346'. 2) Count the distinct types of bonds that can be created by the molecule 'TR346'. This requires querying the 'bond' table where 'molecule_id' equals 'TR346' and counting the distinct 'bond_type'. Both results need to be obtained in a single query. To achieve this, I will use a JOIN operation between the 'atom' and 'bond' tables on the 'molecule_id' column, filtering both tables for 'molecule_id' = 'TR346'. The final query will select the 'atom_id' from the 'atom' table and count the distinct 'bond_type' from the 'bond' table.", 'SQL': "SELECT T1.atom_id, COUNT(DISTINCT T2.bond_type) FROM atom T1 JOIN bond T2 ON T1.molecule_id = T2.molecule_id WHERE T1.molecule_id = 'TR346' AND T2.molecule_id = 'TR346' GROUP BY T1.atom_id"}


2it [00:21, 10.84s/it]

1 :  {'chain_of_thought_reasoning': "To answer the question, I need to find the city of a high school in the county of Merced with specific criteria. First, I'll join the 'schools' and 'frpm' tables on the 'CDSCode' since it's a common key and a foreign key relationship exists between these tables. I'll filter the 'schools' table for schools in Merced County and with an EILCode of 'HS' for high schools. Then, I'll filter the 'frpm' table for records with 'NSLP Provision Status' as 'Lunch Provision 2' and where the lowest grade is 9 and the highest grade is 12. The final step is to select the 'City' from the 'schools' table which meets all these conditions.", 'SQL': "SELECT T1.City FROM schools T1 JOIN frpm T2 ON T1.CDSCode = T2.CDSCode WHERE T1.County = 'Merced' AND T1.EILCode = 'HS' AND T2.`NSLP Provision Status` = 'Lunch Provision 2' AND T2.`Low Grade` = '9' AND T2.`High Grade` = '12'"}


3it [00:31, 10.18s/it]

...

12 :  {'chain_of_thought_reasoning': "To solve the problem, I need to calculate the percentage of bonds that are double bonds ('=') among all bonds in molecule TR047. First, I'll filter the bonds by molecule_id = 'TR047'. Then, I'll count the total number of bonds and the number of double bonds for this molecule. The percentage is calculated by dividing the number of double bonds by the total number of bonds and multiplying by 100 to get a percentage. I'll use a single SQL query with a CASE statement to conditionally count double bonds and use COUNT to count all bonds. The result will be grouped by molecule_id, though this is implicit since we're only considering TR047.", 'SQL': "SELECT (SUM(CASE WHEN bond_type = '=' THEN 1 ELSE 0 END) * 100.0 / COUNT(bond_id)) AS percent FROM bond WHERE molecule_id = 'TR047'"}


14it [02:28, 10.22s/it]

13 :  {'chain_of_thought_reasoning': "To answer the question, I need to find the user who added a bounty of 50 to a post with a title mentioning 'variance'. First, I'll identify the relevant tables and columns: 'votes' for BountyAmount, 'posts' for Title, and 'users' for DisplayName. I'll join these tables based on the relationships defined in the schema: votes.PostId to posts.Id and votes.UserId to users.Id. I'll filter the votes table for BountyAmount = 50 and the posts table for titles containing 'variance'. I'll use a LIKE operator with '%' wildcards to handle any position of 'variance' in the title. Finally, I'll select the DisplayName of the user, ensuring to filter out any null values in the BountyAmount and Title columns as per the admin instructions.", 'SQL': "SELECT T3.DisplayName FROM votes T1 JOIN posts T2 ON T1.PostId = T2.Id JOIN users T3 ON T1.UserId = T3.Id WHERE T1.BountyAmount = 50 AND T2.Title LIKE '%variance%' AND T1.BountyAmount IS NOT NULL AND T2.Title IS NOT NULL

15it [02:38, 10.55s/it]

14 :  {'chain_of_thought_reasoning': "To answer the question, I need to join the 'users' and 'badges' tables based on the user's ID. The 'users' table contains the location information, and the 'badges' table contains the badge acquisition dates. I will filter the 'users' table to include only those users located in 'Rochester, NY'. Then, I will join this filtered list with the 'badges' table to get the dates when these users obtained their badges. I will use aliases for the tables to make the query clearer and ensure that I only select the 'Date' column from the 'badges' table as per the instructions. I will also ensure to filter out any null values in the 'Date' column to comply with the admin instructions.", 'SQL': "SELECT T2.Date FROM users T1 JOIN badges T2 ON T1.Id = T2.UserId WHERE T1.Location = 'Rochester, NY' AND T2.Date IS NOT NULL"}


In [6]:
from pathlib import Path

schema_path = Path('C:/Users/yousf/Bureau/ConvergenceAI/CHESS_Impl/data/test/generated_candidate.json')
with open(schema_path, 'w') as f:
    json.dump(res, f, indent=4)

## Cost Estimation per task

In [5]:

PROMPT_PATH = os.getenv("PROMPT_ROOT_PATH") + "\\candidate_generation.txt"
prompt = load_prompt(PROMPT_PATH)

In [6]:

def tokens_calc(example):
    encoding = tiktoken.get_encoding("cl100k_base")
    num_tokens = len(encoding.encode(example))
    return num_tokens

In [7]:
#prompt template tokens 
tokens_calc(prompt)

631

The prompt template has 631 tokens in total, and we have also 3 other variables (Database_Schema,Question and Hint).
After i see a formatted prompt example it contains about 2000 tokens (because database_schema will be long)
So let's suppose that input tokens is 2000

In [8]:
## Output tokens estimation from an example 
output_example = """
{'chain_of_thought_reasoning': "First, I identified the relevant columns from the question: CDSCode, County Code, School Type, Low Grade, High Grade, and County Name. Then, I determined that the question is asking for the city location of a high school level school with Lunch Provision 2, whose lowest grade is 9 and the highest grade is 12 in the county of Merced. I used the hint that high school can be represented as EILCode = 'HS'. I joined the frpm and schools tables based on the CDSCode, and then filtered the results to match the conditions specified in the question. Finally, I selected the City column from the schools table, which is the column that provides the city location.",
 'SQL': "SELECT City FROM schools WHERE EILCode = 'HS' AND County = 'Merced' AND LowGrade = '9' AND HighGrade = '12' AND NSLPProvisionStatus = 'Lunch Provision 2' AND CDSCode IN (SELECT CDSCode FROM frpm WHERE CountyCode = '02');"}
"""
tokens_calc(output_example)

227

Let's suppose the output tokens is 250

In [9]:
## Price calculation (just with gpt4 because in this module we don't use gpt3.5) 
input_price_per_token_gpt4 = 0.01 / 1000
output_price_per_token_gpt4 = 0.03 / 1000
price_gpt4 = 2000 * input_price_per_token_gpt4 + 250 * output_price_per_token_gpt4
print("estimated price per Task (GPT-4):", price_gpt4, "$")

estimated price per Task (GPT-4): 0.0275 $


In [10]:
## in this module there is a number of retrials so let's estimate the price with different number of retrials 

num_retrials = [1, 2, 3, 4, 5]
for num_retrial in num_retrials:
    total_price = price_gpt4 * num_retrial
    print("The estimated price with number of retrials of " + str(num_retrial) + ": ", total_price)

The estimated price with number of retrials of 1:  0.0275
The estimated price with number of retrials of 2:  0.055
The estimated price with number of retrials of 3:  0.0825
The estimated price with number of retrials of 4:  0.11
The estimated price with number of retrials of 5:  0.1375
